In [11]:
import pandas as pd
import numpy as np
from peewee_models import *


In [12]:

# Transformación y concatenación de los csv anteriormente generados a un solo dataframe para ingestarlos después a la base de datos

wpp_poblacion = pd.read_csv("../datasets/sql/world_population_prospects/wpp_poblacion.csv")
wdi_poblacion = pd.read_csv("../datasets/sql/world_development_indicators/wdi_poblacion.csv")
wpp_poblacion.set_index(["region_id", "year"], inplace=True)
wdi_poblacion.set_index(["region_id", "year"], inplace=True)
table_poblacion = wpp_poblacion.merge(wdi_poblacion, how="right", left_index=True, right_index=True).drop(['id_x','name_x','id_y','name_y'],axis=1).reset_index()
table_poblacion = table_poblacion[table_poblacion['region_id'].notna()]
table_poblacion = table_poblacion.replace(np.nan, None)
table_poblacion.region_id = table_poblacion.region_id.astype(int)

wpp_salud = pd.read_csv("../datasets/sql/world_population_prospects/wpp_salud.csv")
wdi_salud = pd.read_csv("../datasets/sql/world_development_indicators/wdi_salud.csv")
wpp_salud.set_index(["region_id", "year"], inplace=True)
wdi_salud.set_index(["region_id", "year"], inplace=True)
table_salud = wpp_salud.merge(wdi_salud, how="right", left_index=True, right_index=True).drop(['id_x','name_x','id_y','name_y'],axis=1).reset_index()
table_salud = table_salud[table_salud['region_id'].notna()]
table_salud = table_salud.replace(np.nan, None)
table_salud.region_id = table_salud.region_id.astype(int)

wdi_economia = pd.read_csv("../datasets/sql/world_development_indicators/wdi_economia.csv").drop(['id','name'],axis=1)
table_economia = wdi_economia
table_economia = table_economia.replace(np.nan, None)

wdi_educacion = pd.read_csv("../datasets/sql/world_development_indicators/wdi_educacion.csv").drop(['id','name'],axis=1)
table_educacion = wdi_educacion
table_educacion = table_educacion.replace(np.nan, None)

wdi_empleos = pd.read_csv("../datasets/sql/world_development_indicators/wdi_empleos.csv").drop(['id','name'],axis=1)
table_empleos = wdi_empleos
table_empleos = table_empleos.replace(np.nan, None)


In [16]:
table_poblacion = table_poblacion.set_index('region_id')


In [15]:
table_poblacion.columns

Index(['region_id', 'year', 'median_age_as_1_july_years',
       'net_migration_rate_per_1000_population',
       'net_number_migrants_thousands', 'fem_population_as_1_july_thousands',
       'male_population_as_1_july_thousands',
       'urban_population_growth_annual_prcnt',
       'rural_population_growth_annual_prcnt',
       'international_migrant_stock_total', 'population_growth_annual_prcnt',
       'population_in_largest_city'],
      dtype='object')

In [71]:
len(table_poblacion)

4836

In [80]:
Demography.insert(table_poblacion.iloc[4000:5000].to_dict(orient='records')).execute()

IntegrityError: (1452, 'Cannot add or update a child row: a foreign key constraint fails (`henrytest`.`demography`, CONSTRAINT `demography_ibfk_1` FOREIGN KEY (`region_id`) REFERENCES `region` (`id`))')

In [73]:
with database.atomic():
    for idx in range(0, len(table_economia), 100):
        Demography.insert(table_poblacion.iloc[idx:idx+100].to_dict(orient='records')).execute()

IntegrityError: (1452, 'Cannot add or update a child row: a foreign key constraint fails (`henrytest`.`demography`, CONSTRAINT `demography_ibfk_1` FOREIGN KEY (`region_id`) REFERENCES `region` (`id`))')

In [72]:
table_poblacion.iloc[51:4900]

,region_id,year,median_age_as_1_july_years,net_migration_rate_per_1000_population,net_number_migrants_thousands,fem_population_as_1_july_thousands,male_population_as_1_july_thousands,urban_population_growth_annual_prcnt,rural_population_growth_annual_prcnt,international_migrant_stock_total,population_growth_annual_prcnt,population_in_largest_city
51,2,2010,14.404,-5.001,-141.355,13949.295,14240.377,3.779279,2.621229,102246.0,2.894904,3289005.0
52,2,2011,14.679,14.42,418.796,14468.875,14780.282,4.574493,3.412449,0.0,3.689508,3371653.0
53,2,2012,15.011,3.481,105.905,15067.373,15399.105,4.958981,3.798483,0.0,4.077628,3456496.0
54,2,2013,15.239,1.525,48.076,15594.637,15946.572,4.344553,3.185537,0.0,3.466788,3543232.0
55,2,2014,15.501,7.842,255.611,16172.321,16543.889,4.531769,3.374206,0.0,3.657576,3632269.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4831,179,2016,46.316,1.237,156.993,65203.529,61790.329,0.031996,-0.936820,0.0,-0.051137,37326706.0
4832,179,2017,46.721,1.296,164.07,65047.99,61614.482,0.003375,-0.999097,0.0,-0.081874,37397437.0
4833,179,2018,47.139,1.407,177.47,64855.455,61400.411,-0.038428,-1.088373,0.0,-0.126880,37468302.0
4834,179,2019,47.572,1.464,183.953,64633.006,61158.671,-0.051001,-1.123327,0.0,-0.140465,37435191.0


In [82]:
table_poblacion.region_id.unique()

array([  1,   2,   3,   4,   5,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  77,  78,  80,  82,  83,  84,  85,
        86,  87,  88,  90,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161,
       162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 179],
      dtype=int64)

In [67]:
table_poblacion.iloc[21:50].to_dict(orient='records')

[{'region_id': 1,
  'year': 2010,
  'median_age_as_1_july_years': None,
  'net_migration_rate_per_1000_population': None,
  'net_number_migrants_thousands': None,
  'fem_population_as_1_july_thousands': None,
  'male_population_as_1_july_thousands': None,
  'urban_population_growth_annual_prcnt': 3.24435486033937,
  'rural_population_growth_annual_prcnt': 0.130510680872939,
  'international_migrant_stock_total': 61756.0,
  'population_growth_annual_prcnt': 1.06742976658398,
  'population_in_largest_city': 6189370.0},
 {'region_id': 1,
  'year': 2011,
  'median_age_as_1_july_years': None,
  'net_migration_rate_per_1000_population': None,
  'net_number_migrants_thousands': None,
  'fem_population_as_1_july_thousands': None,
  'male_population_as_1_july_thousands': None,
  'urban_population_growth_annual_prcnt': 3.22377949288483,
  'rural_population_growth_annual_prcnt': 0.110106197857633,
  'international_migrant_stock_total': 0.0,
  'population_growth_annual_prcnt': 1.06749342269972,
  

In [49]:
table_poblacion

,region_id,year,median_age_as_1_july_years,net_migration_rate_per_1000_population,net_number_migrants_thousands,fem_population_as_1_july_thousands,male_population_as_1_july_thousands,urban_population_growth_annual_prcnt,rural_population_growth_annual_prcnt,international_migrant_stock_total,population_growth_annual_prcnt,population_in_largest_city
0,1,2020,None,None,None,None,None,2.831514,-0.223584,0.0,0.906299,8602317.0
1,1,1990,None,None,None,None,None,4.008408,1.727181,28118.0,2.185105,3037805.0
2,1,1991,None,None,None,None,None,3.958055,1.670721,0.0,2.138311,3152771.0
3,1,1992,None,None,None,None,None,3.881243,1.594102,0.0,2.070218,3272256.0
4,1,1993,None,None,None,None,None,3.766377,1.485377,0.0,1.968859,3395922.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4831,179,2016,46.316,1.237,156.993,65203.529,61790.329,0.031996,-0.936820,0.0,-0.051137,37326706.0
4832,179,2017,46.721,1.296,164.07,65047.99,61614.482,0.003375,-0.999097,0.0,-0.081874,37397437.0
4833,179,2018,47.139,1.407,177.47,64855.455,61400.411,-0.038428,-1.088373,0.0,-0.126880,37468302.0
4834,179,2019,47.572,1.464,183.953,64633.006,61158.671,-0.051001,-1.123327,0.0,-0.140465,37435191.0


In [87]:
import etl

In [91]:
etl.country_code_df.cod.sort_values().unique().to_list()

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

In [92]:
ccdf = etl.country_code_df

In [98]:
ccdf = ccdf.sort_values('cod').reset_index(drop=True)

In [99]:
ccdf.index = np.arange(1, len(ccdf) + 1)

,cod,name,lang
0,1,vietnam,vi
1,2,afghanistan,fa-AF
2,3,bangladesh,bn-BD
3,4,bhutan,dz
4,5,india,en-IN
...,...,...,...
172,173,hong kong,zh-HK
173,174,macao,zh
174,175,taiwan,zh-TW
175,176,north korea,NaN
